In [ ]:
import os
import sys

# Obtenir le répertoire de travail courant
current_dir = os.getcwd()

# Obtenir le répertoire parent de `maps` (qui est `src`)
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

# Ajouter `src` au chemin de recherche des modules
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [ ]:
import pandas as pd
import numpy as np
import folium.features
from sklearn.cluster import DBSCAN
from sklearn.cluster import HDBSCAN
from sklearn.cluster import OPTICS
from sklearn.linear_model import LinearRegression 
from sklearn.preprocessing import PolynomialFeatures
from pyproj import Transformer
import seaborn as sns

from road_utils import *
from city_utils import plotMapWithColors
from useful_methods.data_processing import extract_data
from useful_methods.neighbours_delaunay.graphs import delaunay_graph
from useful_methods.neighbours_delaunay.miscellaneous_for_neighbouring import mean_distance_choice, mean_distance_to_NN
from useful_methods.ihm.maps.mapUtils import *

In [ ]:
df = pd.read_csv("../../database/data.csv", sep=";", decimal=',')

In [ ]:
df_extracted = extract_data(df, provider='Orange', techno='4g')#, region='Normandie') #Occitanie, Île-de-France
G, pos = delaunay_graph(df_extracted)

In [ ]:
mean_distances = mean_distance_to_NN(df_extracted[['x', 'y']], n_neighbours=3) # 3 to have more neighbours

In [ ]:
cityStationsLabels = []

for station in mean_distances.index:
    if(mean_distances[station]<=1):
        cityStationsLabels.append(station)

In [ ]:
cityLabels = pd.Series(DBSCAN(eps=4000, min_samples=1).fit(df_extracted[['x','y']].loc[cityStationsLabels]).labels_, index = cityStationsLabels)

In [ ]:
cityLabels

In [ ]:
cityBsStationCount = cityLabels.value_counts()

bigCitiesClusters = cityBsStationCount[cityBsStationCount >= 27]
littleCitiesClusters = cityBsStationCount[cityBsStationCount < 27]


In [ ]:
cityBsStationCount[35]

In [ ]:
cityCenters = {cityLabel : (0,0) for cityLabel in cityLabels.unique()}
for bs_id, latitude, longitude in df[['latitude', 'longitude']].loc[cityStationsLabels].itertuples():
    coords = cityCenters.get(cityLabels[bs_id])
    cityCenters[cityLabels[bs_id]] = (latitude + coords[0], longitude + coords[1])

for cityLabel in cityLabels.unique():
    cityCenters[cityLabel] = cityCenters.get(cityLabel) / cityBsStationCount[cityLabel]

In [ ]:
def rgb_to_hex(rgb):
        return '#{:02x}{:02x}{:02x}'.format(int(rgb[0]*255), int(rgb[1]*255), int(rgb[2]*255))

def labelToColor(clustId, clusters, palette):   
    if (clustId == -1):
        return 'gray'
    else:
        return rgb_to_hex(palette[list(clusters).index(clustId)- (1 if -1 in clusters else 0)])

In [ ]:
clusters = cityLabels.unique()
num_clusters = len(clusters) - (1 if -1 in clusters else 0)
palette = sns.color_palette("hsv", num_clusters)
colors = cityLabels.apply(lambda clustId : labelToColor(clustId, clusters, palette))

In [ ]:
map = folium.Map(location=np.mean(df_extracted[['latitude','longitude']].loc[cityStationsLabels], axis=0), zoom_start=7, tiles="Cartodb Positron")
bigCitiesLayer = folium.FeatureGroup(f"Big cities").add_to(map)
littleCitiesLayer = folium.FeatureGroup(f"Little cities").add_to(map)
bigCityCentersLayer = folium.FeatureGroup(f"big cities centers").add_to(map)
liitleCityCenterLayer = folium.FeatureGroup(f"little cities centers").add_to(map)
for bs_id, latitude, longitude in df[['latitude', 'longitude']].loc[cityStationsLabels].itertuples():
    color = colors[bs_id]
    dot = folium.CircleMarker(location=[latitude, longitude], color=color, radius=1, popup=cityLabels[bs_id])

    if(cityLabels[bs_id] in bigCitiesClusters):
        bigCitiesLayer.add_child(dot)
    else:
        littleCitiesLayer.add_child(dot)

for cityLabel in cityLabels.unique():
    dot = folium.CircleMarker(location=cityCenters[cityLabel], color='black', fill_opacity = 1, fill=True, radius=5, popup=cityBsStationCount[cityLabel])
    if(cityLabel in bigCitiesClusters):
        bigCityCentersLayer.add_child(dot)
    else:
        liitleCityCenterLayer.add_child(dot)

folium.LayerControl().add_to(map)

map.save(f"../../out/maps/SeparateLilFromBigCities.html")